In [48]:
# original size 258866 # 258841 # 258816
import sys
import os
from torch.utils.data import DataLoader
sys.path.append("..")
from tqdm import tqdm
import numpy as np
import pandas as pd

from utils import create_metadata_df, measurements_to_df
from dataset_xy import CARLADatasetXY
import json
from send2trash import send2trash
import shutil

# Move sensor folders that we are not using

In [2]:
root_dir = os.path.join("..", "..", "data", "data")
path_data = os.path.join("..", "..", "..", "end2endappras","data", "data")
keep_input = ["lidar", "rgb", "measurements"] # "lidar"

In [3]:
def move_unused_sensors_to_new_folder(root_dir, keep_input):
    for (root, dirs, files) in os.walk(root_dir, topdown=True):
        # Current folder contains the files
        if not dirs:
            dir, input_type = os.path.split(root)
            if input_type not in keep_input:
                path_parts = root.split(os.sep)
                idx_data_first = path_parts.index("data")
                path_parts[idx_data_first + 1] += " unused"
                dir_new = os.path.join(*path_parts)
                if not os.path.exists(dir_new):
                    os.makedirs(dir_new)
                shutil.move(root, dir_new)

# Move folders back into the data folder

In [9]:
# root_dir = os.path.join("..", "..", "data", "data unused")
root_dir = "/Users/julianvonklitzing/Documents/GitHub/end2endappras/data/data unused"

In [10]:
def move():
    for (root, dirs, files) in os.walk(root_dir, topdown=True):
        # Current folder contains the files
        if not dirs:
            dir, input_type = os.path.split(root)
            dir_parts = dir.split("/")
            idx = dir_parts.index("data") # idx of first data
            dir_parts[idx + 1] = "data"
            dir_new = os.path.join(*dir_parts)
            shutil.move(root, dir_new)

# Move/ Delete _prep folders

In [11]:
def move_unused_sensors_to_new_folder(root_dir, keep_input):
    for (root, dirs, files) in os.walk(root_dir, topdown=True):
        # Current folder contains the files
        if not dirs:
            dir, input_type = os.path.split(root)
            if input_type not in keep_input:
                path_parts = root.split(os.sep)
                idx_data_first = path_parts.index("data")
                path_parts[idx_data_first + 1] += " unused"
                dir_new = os.path.join(*path_parts)
                if not os.path.exists(dir_new):
                    os.makedirs(dir_new)
                shutil.move(root, dir_new)

In [49]:
def delete_sensor_folders(root_dir, inputs_to_be_deleted):
    for (root, dirs, files) in os.walk(root_dir, topdown=True):
        # Current folder contains the files
        if not dirs:
            dir, input_type = os.path.split(root)
            if input_type in inputs_to_be_deleted:
                shutil.rmtree(root)

In [50]:
# root_dir = os.path.join("..", "..", "data", "data")
# inputs_to_be_deleted = ["lidar_bev"] # "lidar"
# delete_sensor_folders(root_dir, inputs_to_be_deleted)

# Delete entries with NaN values

In [43]:
path_data = os.path.join("..", "..", "data", "data")

config_xy = {"used_inputs": ["measurements"], 
        "used_measurements": ["speed", "steer", "throttle", "brake", "command"],
        "y": ["brake", "steer", "throttle"],
        "seq_len": 1
        }

# Create df_meta 
df_meta_data = create_metadata_df(path_data, config_xy["used_inputs"])

# Create Dataset & DataLoader
dataset = CARLADatasetXY(root_dir=path_data, df_meta_data=df_meta_data, config=config_xy)
dataloader = DataLoader(dataset=dataset, batch_size=64, num_workers=0, sampler=None, shuffle=False)

In [44]:
df_measurements = measurements_to_df(df_meta_data)

100%|██████████| 258841/258841 [01:49<00:00, 2368.21it/s]


In [45]:
df_measurements_nans = df_measurements.isna()
df_measurements_nans["waypoints"] = df_measurements["waypoints"].apply(lambda x: np.any(np.isnan(np.array(x))))

In [46]:
df_measurements_nans[np.any(df_measurements_nans, axis=1)]

,dir,speed,command,steer,throttle,brake,theta,x,y,x_command,y_command,waypoints


In [40]:
def remove_entries_at_indices(idx_nan_list, df_meta_entire):
    count_files_deleted = 0
    for idx_nan in idx_nan_list:
        path_nan = os.path.join(df_meta_entire.loc[idx_nan]["dir"], "measurements", df_meta_entire.loc[idx_nan]["measurements"])
        with open(path_nan, 'r') as f:
            measurements = json.load(f)
        if not np.isnan(measurements["steer"]):
            print("Break: indexes do not fit!")
            break
        dir = df_meta_entire["dir"].loc[idx_nan]
        dirs_sensors = os.listdir(dir)
        number_entry = df_meta_entire["measurements"].loc[idx_nan].split(".")[0]
        for dir_sensor in dirs_sensors:
            if not dir_sensor.startswith("."):
                files = os.listdir(os.path.join(dir, dir_sensor))
                for file in files:
                    if file.startswith(number_entry):
                        count_files_deleted += 1
                        # os.remove(os.path.join(dir, dir_sensor, file))
                        send2trash(os.path.join(dir, dir_sensor, file))
    return count_files_deleted

In [41]:
# remove_entries_at_indices(idx_nan_list, df_meta_data)

75

## Remove invalid entries of Noise Dataset

In [3]:
root_dir = os.path.join("..", "..", "data", "Noise-Dataset", "Noise-Dataset2")
keep_input = ["lidar", "rgb", "measurements"] # "lidar"

In [4]:
def remove_unused_sensors_to_new_folder(root_dir, keep_input):
    for (root, dirs, files) in os.walk(root_dir, topdown=True):
        # Current folder contains the files
        if not dirs:
            dir, input_type = os.path.split(root)
            if input_type not in keep_input:
                shutil.rmtree(root)

In [5]:
def remove_all_not_rgb_existent_files(root_dir):
    for (root, dirs, files) in os.walk(root_dir, topdown=True):
        # Current folder contains the files
        if not dirs:
            dir, input_type = os.path.split(root)
            if input_type == "rgb":
                file_numbers_rgb = [filename.split(".")[0] for filename in files]
                # Delete measurements files not in rgb
                file_numbers_measurements = [filename.split(".")[0] for filename in os.listdir(os.path.join(dir, "measurements"))]
                for file_number in file_numbers_measurements:
                    if file_number not in file_numbers_rgb:
                        os.remove(os.path.join(dir,"measurements", f"{file_number}.json"))
                # Delete lidar files not in rgb
                file_numbers_lidar = [filename.split(".")[0] for filename in os.listdir(os.path.join(dir, "lidar"))]
                for file_number in file_numbers_lidar:
                    if file_number not in file_numbers_rgb:
                        os.remove(os.path.join(dir,"lidar", f"{file_number}.npy"))                

In [6]:
remove_unused_sensors_to_new_folder(root_dir, keep_input)

In [7]:
remove_all_not_rgb_existent_files(root_dir)

In [8]:
df_meta_data_noisy = create_metadata_df(root_dir, ["rgb", "lidar", "measurements"])

In [9]:
df_meta_data_noisy

,dir,rgb,lidar,measurements
0,../../data/Noise-Dataset/Noise-Dataset2/Town02...,0010.png,0010.npy,0010.json
1,../../data/Noise-Dataset/Noise-Dataset2/Town02...,0011.png,0011.npy,0011.json
2,../../data/Noise-Dataset/Noise-Dataset2/Town02...,0012.png,0012.npy,0012.json
3,../../data/Noise-Dataset/Noise-Dataset2/Town02...,0013.png,0013.npy,0013.json
4,../../data/Noise-Dataset/Noise-Dataset2/Town02...,0014.png,0014.npy,0014.json
...,...,...,...,...
8108,../../data/Noise-Dataset/Noise-Dataset2/Town07...,0035.png,0035.npy,0035.json
8109,../../data/Noise-Dataset/Noise-Dataset2/Town07...,0036.png,0036.npy,0036.json
8110,../../data/Noise-Dataset/Noise-Dataset2/Town07...,0037.png,0037.npy,0037.json
8111,../../data/Noise-Dataset/Noise-Dataset2/Town07...,0038.png,0038.npy,0038.json


In [20]:
df_meta_data = create_metadata_df(root_dir, ["rgb", "lidar", "measurements"])

In [23]:
# Of all data together, the noisy data is only 1.4%
len(df_meta_data_noisy) / (len(df_meta_data) + len(df_meta_data_noisy))

0.014195769835798327